In [1]:
import pandas as pd
import numpy as np

In [168]:
pd.set_option('display.max_columns', None) # display all columns in the dataset
dataset = pd.read_csv(r"C:\Users\ADMIN\Downloads\fifa21 raw data v2.csv")


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6588\584168978.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(r"C:\Users\ADMIN\Downloads\fifa21 raw data v2.csv")


In [173]:
# Remove unnecessary columns
columns_to_drop = ['Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes','Name','photoUrl', 'playerUrl','↓OVA', 'POT', 'Positions','BOV', 'PAC', 'SHO', 'PAS',
       'DRI', 'DEF', 'PHY', 'Hits']

In [174]:
dataset.drop(columns = columns_to_drop, inplace= True)

In [175]:
dataset['LongName'] = dataset['LongName'].str.strip()

In [176]:
dataset['Nationality'] = dataset['Nationality'].str.strip()

In [177]:
dataset['Age'] = dataset['Age'].astype(str).str.strip()

In [178]:
dataset['Age'] = pd.to_numeric(dataset['Age'])

In [179]:
dataset['Club'] = dataset['Club'].replace('\n\n\n\n', '').str.strip()

In [180]:
# Convert the joined column to the correct datetime format 
dataset['Joined'] = pd.to_datetime(dataset['Joined'], format='%d-%b-%y', errors='coerce')

In [181]:
# Extracting the year from the joined column
dataset['Joined Year'] = dataset['Joined'].dt.year

In [182]:
# Function to process contract column
def process_contract(row):
    contract = str(row['Contract']).strip()
    joined_year = row['Joined Year']

    if '~' in contract:  # Standard contract format "YYYY~YYYY"
        start, end = contract.split('~')
        return pd.Series([start.strip(), end.strip(), 'On Contract'])

    elif 'On Loan' in contract:  # Loan format "Month Day, Year On Loan"
        end_year = contract[-9:-5]  # Extract the year from end date
        return pd.Series([joined_year, end_year, 'On Loan'])

    elif 'Free' in contract:  # Free agent
        return pd.Series([joined_year, np.nan, 'Free'])

    else:  # If none of the above (unexpected case)
        return pd.Series([np.nan, np.nan, 'Unknown'])
 


In [183]:
# Apply function to create new columns
dataset[['Contract Start', 'Contract End', 'Contract Status']] = dataset.apply(process_contract, axis=1)

# Convert contract years to numeric format
dataset['Contract Start'] = pd.to_numeric(dataset['Contract Start'], errors='coerce')
dataset['Contract End'] = pd.to_numeric(dataset['Contract End'], errors='coerce')

In [170]:
def convert_height(height):
    if 'cm' in height:
        return int(height.strip('cm'))
    else:
        feet, inches = height.split("'")
        total_inches = int(feet) * 12 + int(inches.strip('"'))
        return round(total_inches * 2.54)

In [ ]:
# Apply the function to the height column
dataset['Height'] = dataset['Height'].apply(convert_height)

In [190]:
# Rename the height column
dataset.rename(columns = {'Height' : 'Height_Cm'}, inplace= True)

In [194]:
# Function to strip away the kgs and lbs from the weight column and convert the pounds to kgs
def convert_weight(weight):
    if 'kg' in weight:
        return int(weight.strip('kg'))
    else:
        pounds = int(weight.strip('lbs'))
        return round(pounds / 2.205)

In [195]:
# Apply the function
dataset['Weight_Kgs'] = dataset['Weight'].apply(convert_weight)

In [205]:
dataset['Preferred Foot'] = dataset['Preferred Foot'].str.strip()

In [211]:
dataset['Best Position'] = dataset['Best Position'].str.strip()

In [221]:
# Function to strip K, M, and € and convert the values with K to millions
def convert_value(value):
    value = str(value).replace('€', '').strip()

    if 'M' in value:
        return float(value.replace('M', '').strip())
    elif 'K' in value:
        return float(value.replace('K', '').strip()) / 1000
    
    return(value)

In [222]:
# Apply the function to the value column
dataset['Value'] = dataset['Value'].apply(convert_value)

In [228]:
dataset.rename(columns = {'Value' : 'Value_in_millions(€)'}, inplace = True)

In [232]:
def convert_wage(wage):
    wage = str(wage).replace("€", "").strip()  # Remove € symbol

    if "K" in wage:  # If wage is in thousands
        return float(wage.replace("K", "").strip())  # Remove 'K' and convert to float

    return float(wage) / 1000  # Convert direct values to thousands 

In [235]:
dataset['Wage'] = dataset['Wage'].apply(convert_wage)

In [237]:
dataset.rename(columns = {'Wage' : 'WeeklyWage_thousands(€)'}, inplace= True)

In [243]:
dataset['Total Stats'] = dataset['Total Stats'].astype(str).str.strip()

In [247]:
dataset['Base Stats'] = dataset['Base Stats'].astype(str).str.strip()

In [257]:
# Function to strip K, M, and € and convert the values with K to millions
def convert_clause(clause):
    clause = str(clause).replace('€', '').strip()

    if 'M' in clause:
        return float(clause.replace('M', '').strip())
    elif 'K' in clause:
        return float(clause.replace('K', '').strip()) / 1000
    
    return(clause)

In [258]:
dataset['Release Clause'] = dataset['Release Clause'].apply(convert_clause)

In [260]:
dataset.rename(columns= {'Release Clause' : 'ReleaseClause_in_millions(€)'}, inplace= True)

In [267]:
dataset['WeakFootRating(/5)'] = dataset['W/F'].str.replace(' ★', '').str.strip()

In [271]:
dataset['WeakFootRating(/5)'] = dataset['WeakFootRating(/5)'].astype(int)

In [276]:
dataset['SM'] = dataset['SM'].str.replace('★', '').str.strip()
dataset['SM'] = dataset['SM'].astype(int)

In [284]:
dataset['A/W'] = dataset['A/W'].str.strip()

In [289]:
dataset['D/W'] = dataset['D/W'].str.strip()

In [292]:
dataset['IR'] = dataset['IR'].str.replace('★', '').str.strip()
dataset['IR'] = dataset['IR'].astype(int)

In [295]:
column_to_drop = ['Contract','Weight','W/F','Joined']
dataset.drop(columns = column_to_drop, inplace= True)

In [305]:
dataset['Contract End'] = dataset['Contract End'].astype(str).str.replace('.0', '')

In [308]:
dataset.drop(columns='Loan Date End', inplace= True)